In [12]:
conda activate bns


Note: you may need to restart the kernel to use updated packages.


In [44]:
import networkx as nx
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
from scipy.stats import lognorm, norm
import os
import copy

from BNS_JT import cpm, variable
from BNS_JT.trans import get_arcs_length, get_all_paths_and_times, get_path_time_idx
from BNS_JT.cpm import variable_elim, mcs_product, single_sample, get_sample_order
from BNS_JT.branch import get_cmat_from_branches, branch_and_bound
from BNS_JT import bnb_fns


In [13]:
np.set_printoptions(precision=3)
HOME = os.getcwd()

# Network

This is a hypothetical network with arbitrary scales, epicentre location and road types.

In [14]:
node_coords = {'n1': (0, 0),
               'n2': (0, -2),
               'n3': (0, -4),
               'n4': (3, 0),
               'n5': (3, -2),
               'n6': (3, -4)} # km

arcs = {'e1': ['n1', 'n2'],
        'e2': ['n1', 'n4'],
        'e3': ['n2', 'n3'],
        'e4': ['n2', 'n5'],
        'e5': ['n3', 'n6'],
        'e6': ['n4', 'n5'],
        'e7': ['n5', 'n6']}

# Fragility curves -- From HAZUS-EQ model (roads are regarded as disconnected when being extensively or completely damaged)

frag = {'major': {'med': 60.0, 'std': 0.7},
        'urban' : {'med': 24.0, 'std': 0.7},
        'bridge': {'med': 1.1, 'std': 3.9}}

arcs_type = {'e1': 'major',
             'e2': 'bridge',
             'e3': 'urban',
             'e4': 'bridge',
             'e5': 'major',
             'e6': 'urban',
             'e7': 'major'}

arcs_avg_kmh = {'e1': 50,
                'e2': 50,
                'e3': 30,
                'e4': 50,
                'e5': 50,
                'e6': 30,
                'e7': 50}

var_ODs = {'od1': ('n1', 'n4'),
           'od2': ('n1', 'n5'),
           'od3': ('n1', 'n6')}

nODs = len(var_ODs)

# Arcs' states index compatible with variable B index, and C
arc_surv = 1 - 1
arc_fail = 2 - 1
arc_either = 3 - 1

arc_lens_km = get_arcs_length(arcs, node_coords)

# Distance to epicentre (epicentre is assumed to have been observed.)
arcs_cloc_km = {}
for k, v in arcs.items():
    cloc = 0.5 * (np.array( node_coords[v[0]] ) + np.array( node_coords[v[1]] ))
    arcs_cloc_km[k] = cloc

epi_loc_km = (20, -3)
arcs_epi_dist_km = {}
for k, v in arcs_cloc_km.items():
    diff = np.array(v) - np.array(epi_loc_km)
    arcs_epi_dist_km[k] = np.sqrt(np.sum(diff**2))
print(arcs_epi_dist_km)

#arcTimes_h = arcLens_km ./ arcs_Vavg_kmh
arc_times_h = {k: v/arcs_avg_kmh[k] for k, v in arc_lens_km.items()}

# create a graph
G = nx.Graph()
for k, x in arcs.items():
    G.add_edge(x[0], x[1], time=arc_times_h[k], label=k)

for k, v in node_coords.items():
    G.add_node(k, pos=v)

path_time = get_all_paths_and_times(var_ODs.values(), G, key='time')
print( path_time )

# plot graph
pos = nx.get_node_attributes(G, 'pos')
edge_labels = nx.get_edge_attributes(G, 'label')

fig = plt.figure()
ax = fig.add_subplot()
nx.draw(G, pos, with_labels=True, ax=ax)
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, ax=ax)
fig.savefig( os.path.join(HOME, 'graph_toy.png'), dpi=200)

{'e1': 20.09975124224178, 'e2': 18.741664813991314, 'e3': 20.0, 'e4': 18.527007313648905, 'e5': 18.527007313648905, 'e6': 17.11724276862369, 'e7': 17.0}
{('n1', 'n4'): [(['e1', 'e3', 'e5', 'e7', 'e6'], 0.2733333333333333), (['e1', 'e4', 'e6'], 0.16666666666666669), (['e2'], 0.06)], ('n1', 'n5'): [(['e1', 'e3', 'e5', 'e7'], 0.20666666666666667), (['e1', 'e4'], 0.1), (['e2', 'e6'], 0.12666666666666665)], ('n1', 'n6'): [(['e1', 'e3', 'e5'], 0.16666666666666666), (['e1', 'e4', 'e7'], 0.14), (['e2', 'e6', 'e4', 'e3', 'e5'], 0.3133333333333333), (['e2', 'e6', 'e7'], 0.16666666666666666)]}


In [15]:
print( path_time )

{('n1', 'n4'): [(['e1', 'e3', 'e5', 'e7', 'e6'], 0.2733333333333333), (['e1', 'e4', 'e6'], 0.16666666666666669), (['e2'], 0.06)], ('n1', 'n5'): [(['e1', 'e3', 'e5', 'e7'], 0.20666666666666667), (['e1', 'e4'], 0.1), (['e2', 'e6'], 0.12666666666666665)], ('n1', 'n6'): [(['e1', 'e3', 'e5'], 0.16666666666666666), (['e1', 'e4', 'e7'], 0.14), (['e2', 'e6', 'e4', 'e3', 'e5'], 0.3133333333333333), (['e2', 'e6', 'e7'], 0.16666666666666666)]}


# Add and sample hazard nodes.

We assume magnitude and epicentre are observed, and sample intensity measures (IMs) experienced by each component event. Note that such samples can be obtained by any approaches (e.g. ground motion prediction equations (GMPEs) or detailed simulation tools like Global Earthquake Model (GEM)). <br>
Here, we employ a GMPE proposed by Campbell (1997) for peak ground acceleration (PGA) and HAZUS EQ model for permanent ground displacement (PGD).

In [16]:
eq_m = 7 # observed

arcs_lpga_mean = {}
for k, v in arcs_epi_dist_km.items():
    lpga_mean = -3.512 + 0.904 * eq_m - 1.312 * np.log( np.sqrt( v**2 + (0.149 * np.exp(0.647*eq_m))**2 ) ) # Campbell (1997)
    arcs_lpga_mean[k] = lpga_mean
print( 'E[ log(A_H) ] for each arc (component):' )
print( {k: np.exp(v) for k,v in arcs_lpga_mean.items()} )

E[ log(A_H) ] for each arc (component):
{'e1': 0.2529664900800276, 'e2': 0.26886113914642845, 'e3': 0.2540890253419107, 'e4': 0.2714972127053331, 'e5': 0.2714972127053331, 'e6': 0.28968581274095156, 'e7': 0.2912683553162252}


We get the samples of PGA. Note that this is not an entirely correct application of Campbell (1997). Procedure is simplified to just to quickly get samples of IM.

In [18]:
no_samp = 10
no_arc = len(arcs)

cpms = {}
varbs = {}

C_pga = np.empty(shape=(no_samp, no_arc))
for i in range(no_arc):
    lpga_mean = arcs_lpga_mean['e'+str(i+1)]
    lpga_samps = np.random.normal( loc = lpga_mean, scale = 0.55, size = (no_samp,) )
    pga_samps = np.exp( lpga_samps )
    C_pga[:,i] = pga_samps
print(C_pga)


# TODO: Below does not work. CPM needs to be able to be defined over continuous or sampled variables.
#varbs['pga'] = variable.Variable(name='pga', B = [], values = [])
#p = np.empty(shape = (no_samp, 1))
#q = np.empty(shape = (no_samp, 1))
#cpms['pga'] = cpm.Cpm( variables = [varbs['pga']], no_child = no_arc, C = C_pga, p = p, q = q )
#print(varbs['pga'])
#print(cpms['pga'])


[[0.079 0.152 0.131 0.201 0.227 0.217 0.377]
 [0.306 0.57  0.676 0.102 0.343 0.364 0.36 ]
 [0.103 0.231 0.365 0.136 0.424 0.322 0.198]
 [0.145 0.49  0.373 0.131 0.07  0.114 0.247]
 [0.257 0.254 0.127 0.198 0.33  0.194 0.312]
 [0.298 0.371 0.312 0.073 1.403 0.712 0.324]
 [0.425 0.468 0.644 0.256 0.172 0.204 0.187]
 [0.17  0.148 0.229 0.441 0.114 0.9   0.319]
 [0.135 0.438 0.157 0.212 0.191 0.213 0.206]
 [0.182 0.61  0.168 0.453 0.169 0.15  0.13 ]]


Sample PGD. 

For PGD, which is the IM for paved roads, the causal mechanism is assumed as lateral spreading due to liquefaction (ref: HAZUS EQ model). <br>
Uncerty in PGD arises from PGA. For the moment, uncertainty in susceptibility category, and susceptibility category is assumed to be a single category (ref: Table 4-12 in HAZUS EQ model is ignored).

In [19]:
pga_c = 0.21 # (g) for Low

k_del = 0.0086 * eq_m**3 - 0.0914*eq_m**2 + 0.4698*eq_m - 0.9835

C_pgd = np.empty(shape=(no_samp, no_arc))
for i in range(no_arc):
    for j in range(no_samp):
        pga = C_pga[j,i]

        pga_rat = pga / pga_c
        if pga_rat < 1:
            pgd = 0
        elif pga_rat < 2:
            pgd = 12*pga_rat - 12 # inch
        elif pga_rat < 3:
            pgd = 18*pga_rat - 24
        else:
            pgd = 70*pga_rat - 180
        
        C_pgd[j,i] = k_del * pgd
        
print(C_pgd)


[[0.000e+00 0.000e+00 0.000e+00 0.000e+00 7.609e-01 2.927e-01 7.397e+00]
 [4.241e+00 1.928e+01 3.506e+01 0.000e+00 5.898e+00 6.815e+00 6.651e+00]
 [0.000e+00 9.127e-01 6.859e+00 0.000e+00 9.578e+00 4.976e+00 0.000e+00]
 [0.000e+00 1.396e+01 7.232e+00 0.000e+00 0.000e+00 0.000e+00 1.663e+00]
 [2.081e+00 1.971e+00 0.000e+00 0.000e+00 5.323e+00 0.000e+00 4.529e+00]
 [3.887e+00 7.122e+00 4.539e+00 0.000e+00 2.234e+02 4.442e+01 5.051e+00]
 [9.623e+00 1.248e+01 2.695e+01 2.037e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 8.543e-01 1.068e+01 0.000e+00 9.328e+01 4.824e+00]
 [0.000e+00 1.054e+01 0.000e+00 9.368e-02 0.000e+00 1.387e-01 0.000e+00]
 [0.000e+00 2.197e+01 0.000e+00 1.152e+01 0.000e+00 0.000e+00 0.000e+00]]


Now we assume that we observed PGA at e2 and e5 by 0.2g and 0.3g respectively. We also assume that there is 10% of observation error. <br>
This updates sample weights by $f(oa_2 | a_2) \cdot f(oa_5 | a_5)$.

In [20]:
q = np.ones(shape=(no_samp,1)) # sample weight vector

for j in range(no_samp):
    q_j = norm.pdf( 0.2, C_pga[j, 2], scale = 0.2*0.1 ) * norm.pdf( 0.3, C_pga[j, 5], scale = 0.3*0.1 )
    q[j] = q_j
    
print(q)

[[1.486e-002]
 [5.045e-122]
 [3.933e-013]
 [6.288e-023]
 [6.180e-004]
 [4.889e-046]
 [1.224e-107]
 [9.175e-086]
 [4.081e-001]
 [2.671e-004]]


# Now component and system events.

First, we compute P(X_n=fail | sample_j).

In [21]:
arcs_pf = {}

for i in range(no_arc):
    k = 'e' + str(i+1)
    pf = np.zeros(shape=(no_samp,1))
    
    for j in range(no_samp):
        _type = arcs_type[k]
        if _type == 'bridge':
            prob = lognorm.cdf(C_pga[j,i], frag[_type]['std'], scale=frag[_type]['med'])
        else:
            prob = lognorm.cdf(C_pgd[j,i], frag[_type]['std'], scale=frag[_type]['med'])
        pf[j] = prob
        
    arcs_pf[k] = pf

    
print(arcs_pf['e2'])


[[0.306]
 [0.433]
 [0.344]
 [0.418]
 [0.354]
 [0.39 ]
 [0.413]
 [0.303]
 [0.407]
 [0.44 ]]


Quantify system (functionality) events.

In [28]:
print(var_ODs[k])

('n1', 'n4')


In [35]:
print(len(path_time[var_ODs[k]]))
print(path_time[var_ODs[k]])

times = [x[1] for x in path_time[var_ODs[k]]]
times.sort()
times.insert(0, np.inf)
print(times)

3
[(['e1', 'e3', 'e5', 'e7', 'e6'], 0.2733333333333333), (['e1', 'e4', 'e6'], 0.16666666666666669), (['e2'], 0.06)]
[inf, 0.06, 0.16666666666666669, 0.2733333333333333]


In [ ]:
i = 0 # System event index
k = 'od' + str(i+1)
no_path = len(path_time[var_ODs[k]])
variables[k] = variable.Variable(name=k, B=np.eye(no_path), values=['Fail', 'Surv'])

In [42]:
variables = {}

i = 0 # System event index
k = 'od' + str(i+1)

times = [x[1] for x in path_time[var_ODs[k]]]
times.sort(reverse=True)
times.insert(0, np.inf)
no_path = len(times)
print(times)

variables[k] = variable.Variable(name=k, B=np.eye(no_path), values=times)
path_time_idx = get_path_time_idx( path_time[var_ODs[k]], variables[k])

print(path_time_idx)

[inf, 0.2733333333333333, 0.16666666666666669, 0.06]
[([], inf, 0), (['e2'], 0.06, 3), (['e1', 'e4', 'e6'], 0.16666666666666669, 2), (['e1', 'e3', 'e5', 'e7', 'e6'], 0.2733333333333333, 1)]


In [49]:
"""
comp_max_states = 2

paths = [x[0] for x in path_time_idx]
paths.remove([])

info = {'path': [[2], [3, 1]],
        'time': np.array([0.0901, 0.2401]),
        'arcs': [i for i in range(1,len(arcs))],
        'max_state': comp_max_states
       }

branches = branch.run_bnb(sys_fn=bnb_fns.bnb_sys,
                       next_comp_fn=bnb_fns.bnb_next_comp,
                       next_state_fn=bnb_fns.bnb_next_state,
                       info=info,
                       comp_max_states=comp_max_states)
"""

"\ncomp_max_states = 2\n\npaths = [x[0] for x in path_time_idx]\npaths.remove([])\n\ninfo = {'path': [[2], [3, 1]],\n        'time': np.array([0.0901, 0.2401]),\n        'arcs': [i for i in range(1,len(arcs))],\n        'max_state': comp_max_states\n       }\n\nbranches = branch.run_bnb(sys_fn=bnb_fns.bnb_sys,\n                       next_comp_fn=bnb_fns.bnb_next_comp,\n                       next_state_fn=bnb_fns.bnb_next_state,\n                       info=info,\n                       comp_max_states=comp_max_states)\n"

In [43]:
lower = {x: 0 for x in arcs}  # Fail
upper = {x: 1 for x in arcs}  # surv
arc_condn = 1


#######TODO: branch_and_bound no longer works#################
sb = branch_and_bound(path_time_idx, lower, upper, arc_condn)
############################################################

B = np.array([[1, 0], [0, 1], [1, 1]])
for i in range(1, 8):
    variables[f'e{i}'] = variable.Variable(name=f'e{i}', B=B, values=['Fail', 'Surv'])
Cmat = get_cmat_from_branches(sb, variables)
print(Cmat)

TypeError: branch_and_bound() takes 3 positional arguments but 4 were given

Calculate a probability vector correponding to Cmat.

In [ ]:
no_c = len(Cmat)
p = np.ones(shape = (no_c, no_samp))

for i in range(no_arc):
    k = f'e{i+1}'
    pf_i = arcs_pf[k]
    
    for j in range(no_samp):

        for k in range(no_c):
            C_ik = Cmat[k,i+1]
            if C_ik == 0:
                prob_ijk = pf_i[j]
            elif C_ik == 1:
                prob_ijk = 1 - pf_i[j]
            else:
                prob_ijk = 1

            p[k,j] *= prob_ijk
    
print(p)

System probability of OD1.

In [ ]:
def cal_sys_probs( Cmat, p ):

    denom = np.sum( p ) # denominator of importance sampling
    numers = np.sum(p, axis=1)

    no_sys_st = max([x[0] for x in Cmat])
    sys_probs = np.empty(shape=(no_sys_st+1,))
    for i in range(no_sys_st+1):
        pos = np.argwhere(Cmat[:,0]==i)
        prob = np.sum(numers[pos]) / denom
        sys_probs[i] = prob
        
    return sys_probs


In [ ]:
sys_probs = cal_sys_probs( Cmat, p )
print(sys_probs)

# Updating system events by observations on component events.

Suppose e1 and e3 are observed to be failed. There is 90% of probability that an observation is correct (i.e. observation error of 10%).

In [ ]:
ox = {'e1': 0, 'e6': 0}
ox_corr = 0.9
c_st = 2 # composite state (representing either of both states)

p_ox = copy.deepcopy(p)
for k, v in ox.items():
    arc_ind = int(k[1])
    
    for i in range(no_c):
        if Cmat[i,arc_ind] == v:
            p_ox[i] *= ox_corr
        elif Cmat[i,arc_ind] != c_st:
            p_ox[i] *= (1-ox_corr)

            
sys_probs_ox = cal_sys_probs( Cmat, p_ox )
print(sys_probs_ox)     

# Updating component events by observations on system events.

Suppose od1 is observed to be delayed. We have probabilities of [0.2, 0.4, 0.4, 0].

In [ ]:
of = {'od1': [0.2, 0.4, 0.4, 0]}

p_of = copy.deepcopy(p)
for k, v in of.items():
    arc_ind = int( k[2] )
    
    for i in range( no_c ):
        p_of[i] *= v[Cmat[i,0]]


In [ ]:
def cal_comp_probs( Cmat, p, comp_ind ):

    numers = np.sum(p, axis=1)

    no_st = 2 # fail or survival (multi-state is not considered at the moment)
    probs = np.empty(shape=(no_st,))
    for i in range(no_st):
        pos = np.argwhere(Cmat[:,comp_ind+1]==i)
        prob = np.sum(numers[pos]) 
        probs[i] = prob
    
    probs = probs / np.sum(probs)
    return probs

In [ ]:
probs_of_x1 = cal_comp_probs( Cmat, p_of, 1 )
print(probs_of_x1)

probs_of_x2 = cal_comp_probs( Cmat, p_of, 2 )
print(probs_of_x2)

probs_of_x3 = cal_comp_probs( Cmat, p_of, 3 )
print(probs_of_x3)

probs_of_x4 = cal_comp_probs( Cmat, p_of, 4 )
print(probs_of_x4)